First, let's embed a ring in 3D.

In [1]:
import os
import sys
import matplotlib.pyplot as plt
import multiprocessing as mp
import numpy as np
from scipy import integrate
import torch

sys.path.append(os.path.join(os.getenv("HOME"), "RNN_Manifold/"))
from manifold_encoder_decoder import s1_direct_product_decoder, s1_direct_product_generator, geometry_util

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [3]:
encoder, decoder = s1_direct_product_generator.train(1, 3, device, n_training_iterations=3000)

angles = np.arange(start=0, stop=2 * np.pi, step=0.01)
with torch.no_grad():
    points = geometry_util.torch_angles_to_ring(torch.tensor(angles, dtype=torch.get_default_dtype()).to(device))
    points = torch.unsqueeze(points, -2)
    test_embedding = encoder(points)
test_embedding = test_embedding.cpu().numpy()


/home/trevor/brainvenv/lib/python3.8/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


iteration: 0, decoding loss: 0.9843466877937317, distance cost: 0.013388672843575478
iteration: 2, decoding loss: 0.5395281314849854, distance cost: 0.029766438528895378
iteration: 3, decoding loss: 0.3357117474079132, distance cost: 0.0691237673163414
iteration: 4, decoding loss: 0.2878531515598297, distance cost: 0.10309571772813797
iteration: 44, decoding loss: 0.3322509825229645, distance cost: 0.03150106966495514
iteration: 46, decoding loss: 0.30295056104660034, distance cost: 0.028324997052550316
iteration: 47, decoding loss: 0.29810237884521484, distance cost: 0.028305288404226303
iteration: 48, decoding loss: 0.2747058868408203, distance cost: 0.02693209983408451
iteration: 50, decoding loss: 0.2715630829334259, distance cost: 0.0279324259608984
iteration: 51, decoding loss: 0.2570587396621704, distance cost: 0.0338280089199543
iteration: 52, decoding loss: 0.23910722136497498, distance cost: 0.028773006051778793
iteration: 53, decoding loss: 0.22587983310222626, distance cost

The shape it generates should be pretty much random, as there is nothing in the loss function we use that encourages a specific shape

In [4]:
%matplotlib tk
proj_fig = plt.figure()
proj_axs = proj_fig.add_subplot(projection="3d")
proj_axs.scatter(test_embedding[:, 0], test_embedding[:, 1], test_embedding[:, 2], cmap="hsv", c=angles)


Now let's generate a higher dimensional ring

In [ ]:
encoder, decoder = s1_direct_product_generator.train(1, 24, device, n_training_iterations=3000)

angles = np.arange(start=0, stop=2 * np.pi, step=0.01)
with torch.no_grad():
    points = geometry_util.torch_angles_to_ring(torch.tensor(angles, dtype=torch.get_default_dtype()).to(device))
    points = torch.unsqueeze(points, -2)
    high_d_generated_ring_data = encoder(points)
high_d_generated_ring_data = high_d_generated_ring_data.cpu().numpy()


iteration: 0, decoding loss: 1.0117884874343872, distance cost: 0.002101421821862459
iteration: 1, decoding loss: 0.49193379282951355, distance cost: 0.04621131345629692
iteration: 2, decoding loss: 0.36218971014022827, distance cost: 0.06344244629144669
iteration: 3, decoding loss: 0.3369177281856537, distance cost: 0.08562996238470078
iteration: 5, decoding loss: 0.28197726607322693, distance cost: 0.12151344865560532
iteration: 6, decoding loss: 0.2761057913303375, distance cost: 0.10752193629741669
iteration: 7, decoding loss: 0.2822996973991394, distance cost: 0.09752345830202103
iteration: 10, decoding loss: 0.32603174448013306, distance cost: 0.021332481876015663


And then decode it

In [ ]:
high_d_generated_ring_data = high_d_generated_ring_data/np.mean(np.abs(high_d_generated_ring_data))

In [ ]:
encoder, decoder = s1_direct_product_decoder.train(data=high_d_generated_ring_data, manifold_dim=1, device=device,
                                                   n_training_iterations=3000, decoder_weight=10, order_red_weight=0.1)


In [ ]:
with torch.no_grad():
    decoded_points, decoded_angles = decoder(torch.tensor(high_d_generated_ring_data, dtype=torch.get_default_dtype()).to(device))

predicted_phases = torch.squeeze(decoded_angles).cpu().numpy()

In [ ]:
def reference_phases(phases):
    phases_refd = phases - phases[0]
    phases_refd = np.arctan2(np.sin(phases_refd), np.cos(phases_refd))
    return phases_refd * np.sign(phases_refd[1])

In [ ]:
def compare_to_ground_truth(predicted_phases, ground_truth_phases, plot_ax):
    refd_test_phases = reference_phases(predicted_phases)
    refd_true_phases = reference_phases(ground_truth_phases)
    line = np.arange(start=-np.pi, stop=np.pi, step=0.01)
    plot_ax.scatter(refd_true_phases, refd_test_phases)
    plot_ax.plot(line, line, color="black", linestyle="--", label="y=x")
    plot_ax.set_xlabel("True Phase")
    plot_ax.set_ylabel("Found Phase")
    return refd_test_phases, refd_true_phases


In [ ]:
%matplotlib inline
fig, ax = plt.subplots()
refd_predicted_phases, refd_true_phases = compare_to_ground_truth(predicted_phases, angles, ax)


Now let's do a harder problem. Let's decode noisy data generated by a dynamic system. First, the dynamics. We will use a typical ring attractor model,

In [ ]:
def conv_circ(signal, ker):
    return np.fft.ifft(np.einsum("ij, j -> ij", np.fft.fft(signal, axis=1), np.fft.fft(ker)), axis=1)


def cosine_kernel(w_0, w_1, N):
    step = 2/(N)
    grid = np.arange(start=0, stop=2, step=step) * np.pi
    weights = -w_0 + w_1 * np.cos(grid)
    return weights


def ring_attractor_dynamics(state, kernel, bias_vec, nonlin_fn):
    if len(np.shape(state)) == 1:
        state = np.expand_dims(state, -1)
    state = np.transpose(state)
    return np.transpose(-state + nonlin_fn(conv_circ(state, kernel) + bias_vec))


def sigmoid(x):
    return np.exp(x)/(np.exp(x) + 1)

Generate some samples of equalibrium states. We will generate around 50, to make things interesting.

In [ ]:
N = 2 ** 7
n_samples = 50

init_conds = np.random.uniform(-1, 1, (n_samples, N)).astype(np.complex)

w_0 = 1
w_1 = 1
kernel = cosine_kernel(w_0, w_1, N)


def run_equilibriation(init_conds):
    soln = integrate.solve_ivp(lambda _, y: ring_attractor_dynamics(y, kernel, np.zeros(N), sigmoid),
                               [0, 20], init_conds, vectorized=True)
    return soln.y[:, -1]


p = mp.Pool()
solns = np.array(p.map(run_equilibriation, init_conds))
p.close()
p.join()

Add some noise

In [ ]:
ring_attractor_data = np.real(solns/np.mean(np.abs(solns)))
noisy_ring_attractor_data = ring_attractor_data + np.random.normal(0, np.max(ring_attractor_data)/15, np.shape(ring_attractor_data))

Note down the ground truth phases

In [ ]:
true_ring_att_phases = (np.argmax(ring_attractor_data, axis=1)/N) * 2 * np.pi

In [ ]:
fig, ax = plt.subplots()
ax.imshow(noisy_ring_attractor_data, aspect="auto")
ax.set_title("Raw activation data")
ax.set_xlabel("Neuron Index")
ax.set_ylabel("Datapoint")


Now decode

In [ ]:
encoder, decoder = s1_direct_product_decoder.train(data=noisy_ring_attractor_data, manifold_dim=1, device=device,
                                                   n_training_iterations=3000, decoder_weight=10, order_red_weight=0.1)


In [ ]:
with torch.no_grad():
    decoded_points, decoded_angles = decoder(torch.tensor(noisy_ring_attractor_data, dtype=torch.get_default_dtype()).to(device))

ring_att_predicted_phases = torch.squeeze(decoded_angles).cpu().numpy()

In [ ]:
%matplotlib inline
fig, ax = plt.subplots()
refd_ring_att_predicted_phases, refd_ring_att_true_phases = compare_to_ground_truth(ring_att_predicted_phases, true_ring_att_phases, ax)
